### **Prétraitement des Données :**

#### **1. Identification et Traitement des Valeurs Manquantes :**

Dans notre dataset, les colonnes : ``Glucose``, ``BloodPressure``, ``SkinThickness``, ``Insulin``, ``BMI`` contiennent des zéros qui ne sont pas des vraies valeurs, mais des valeurs manquantes (parce qu’un patient ne peut pas avoir un glucose ou un BMI = 0).

**Pourquoi ces zéros sont considérés comme manquants ?**

Dans notre dataset :

- ``0`` a été mis à la place des valeurs non mesurées.

- Ces zéros faussent les statistiques (moyenne, écart-type, corrélation…).

- Si on ne les traite pas, notre modèle par la suite va croire que ``0`` est une vraie valeur physiologique, ce qui peut détourner complètement les clusters.

##### **1.1. Remplaçement des Zéros avec NaN :**



In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/raw/data.csv")

columns = data.columns.to_list()

delete = columns.pop(0)

data_copy = pd.DataFrame(data, columns=columns)

cols_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

data_copy[cols_with_zeros] = data_copy[cols_with_zeros].replace(0, np.nan)

data_copy.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33


##### **1.2. Imputation avec KNN Imputer :**

Le ``KNN Imputer`` (``K-Nearest Neighbors Imputer``) est une méthode d’imputation des valeurs manquantes basée sur l’algorithme des K plus proches voisins (``KNN``).

Son rôle est de remplacer les valeurs manquantes dans un jeu de données par des valeurs estimées à partir des observations les plus similaires (les voisins).

**Comment ça fonctionne ?**

1. Pour chaque observation contenant une ou plusieurs valeurs manquantes : On cherche les ``K observations`` les plus proches (en termes de distance).

2. Pour trouver cette similarité, le ``KNN Imputer`` calcule la distance (souvent ``euclidienne``) entre les lignes en ignorant les colonnes manquantes.

3. Ensuite, pour chaque valeur manquante : Il remplace la valeur manquante par la ``moyenne`` (ou ``médiane``) des valeurs correspondantes trouvées chez les ``K`` voisins.

**la distance Euclidienne :**

si on a deux points A et B dans un espace à 2 dimensions : $A(x_1, y_1)$ et $B(x_2, y_2)$

Alors la distance euclidienne $d(A,B)$ est :

$$d(A,B) = \sqrt{(x_1 - x_2)² + (y_1 - y_2)²}$$

In [13]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

data_copy[cols_with_zeros] = imputer.fit_transform(data_copy[cols_with_zeros])

data = data_copy

print(data.isnull().sum().sum())

data.head()

0


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,209.0,33.6,0.627,50
1,1,85.0,66.0,29.0,73.0,26.6,0.351,31
2,8,183.0,64.0,29.4,383.2,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33
